In [1]:
import pandas as pd
import re

### 1. Para leer correctamente las columnas se necesita que el argumento *sep* sea igual a "*;*".

In [2]:
dolar_2019 = pd.read_csv("../data/raw/dolar_2019_sii.csv", sep = ";")

In [3]:
dolar_2019.head()

,Día,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,1,NaN,"657,81","651,79","678,53",NaN,NaN,"679,15","700,82",NaN,"728,21",NaN,NaN
1,2,"694,77",NaN,NaN,"672,56","678,71",NaN,"677,62","704,88","720,20","729,38",NaN,"812,13"
2,3,"697,09",NaN,NaN,"670,05","683,31","709,80","681,30",NaN,"724,72","727,80",NaN,"804,29"
3,4,"697,64","655,39","656,57","667,55",NaN,"704,59","680,48",NaN,"725,69","719,53","735,05","802,44"
4,5,NaN,"653,80","659,07","666,32",NaN,"696,78","677,64","711,13","723,30",NaN,"736,66","796,75"


### 2. dolar_2020_sii.txt

Diferencias:
- *dolar_2020_sii.txt* no tiene los nombres de las columnas.
- El separador es distinto (|).
- La extensión de los archivos es distinta (.txt)

Similitudes:
- Ambos presentan los valores del dólar por día del mes.
- Ambos presentan valores numéricos reales con 2 decimales.
- Ambos incluyen al final una fila con los promedios por mes.
- Ambos presentan datos para 31 días. 

In [4]:
dolar_2020_txt = pd.read_csv("../data/raw/dolar_2020_sii.txt", sep = "|", header = None)

Para leer correctamente las columnas se necesita que el argumento *sep* sea igual a "|" y agregar el argumento *header* con un valor igual a *None*.

In [5]:
dolar_2020_txt.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,NaN,NaN,NaN,"852,03",NaN,"806,32","821,23",NaN,"776,46","788,15",NaN,"767,29"
1,2,"748,74",NaN,"818,32","863,81",NaN,"796,46","816,29",NaN,"770,59","785,04","771,92","760,16"
2,3,"754,16","799,11","817,45","861,89",NaN,"782,86","803,98","757,06","771,45",NaN,"769,17","755,34"
3,4,NaN,"790,92","809,81",NaN,"837,92","769,13",NaN,"761,34","773,71",NaN,"758,53","752,03"
4,5,NaN,"781,78","812,19",NaN,"838,74","775,58",NaN,"767,98",NaN,"790,81","757,16",NaN


### 3. El archivo dolar_2020_sii_AA.dat contiene la misma información que el archivo anterior pero esta vez con valores separados por combinaciones de 3 letras.

In [6]:
from io import StringIO
data = open(r"../data/raw/dolar_2020_sii_AA.dat", "r").read()
regex = r'[a-zA-Zá-ý]{3}'
for line in re.findall(regex, data):
    if line != "Pro" and line != "med" and line != "ioX":
        data = re.sub(line, ";", data)
data = StringIO(data)
dolar_2020_regex = pd.read_csv(data, sep=';', header = None)

### 4. Wrangling y limpieza

In [7]:
dolar_2019 = dolar_2019.drop(axis = 0, index = 31)

dolar_2020_txt = dolar_2020_txt.drop(axis = 0, index = 31)
dolar_2020_txt.columns = list(dolar_2019.columns)

dolar_2020_regex = dolar_2020_regex.drop(axis = 0, index = 31)
dolar_2020_regex.columns = list(dolar_2019.columns)

In [8]:
dolar_2019.head()
# dolar_2020_txt.head()
# dolar_2020_regex.head()

,Día,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,1,NaN,"657,81","651,79","678,53",NaN,NaN,"679,15","700,82",NaN,"728,21",NaN,NaN
1,2,"694,77",NaN,NaN,"672,56","678,71",NaN,"677,62","704,88","720,20","729,38",NaN,"812,13"
2,3,"697,09",NaN,NaN,"670,05","683,31","709,80","681,30",NaN,"724,72","727,80",NaN,"804,29"
3,4,"697,64","655,39","656,57","667,55",NaN,"704,59","680,48",NaN,"725,69","719,53","735,05","802,44"
4,5,NaN,"653,80","659,07","666,32",NaN,"696,78","677,64","711,13","723,30",NaN,"736,66","796,75"


In [9]:
dolar_2019["year"] = 2019
dolar_2020_txt["year"] = 2020
dolar_2020_regex["year"] = 2020

In [10]:
for column in dolar_2019.columns[1:-1]:
    dolar_2019[column] = dolar_2019[column].apply(lambda x: str(x).replace(',', '.')).astype(float)
    dolar_2020_txt[column] = dolar_2020_txt[column].apply(lambda x: str(x).replace(',', '.')).astype(float)
    dolar_2020_regex[column] = dolar_2020_regex[column].apply(lambda x: str(x).replace(',', '.')).astype(float)

In [11]:
dolar_2019["Día"] = dolar_2019["Día"].astype(int)
dolar_2020_txt["Día"] = dolar_2020_txt["Día"].astype(int)
dolar_2020_regex["Día"] = dolar_2020_regex["Día"].astype(int)

### 5. Exportar archivos

In [12]:
dolar_2019.to_csv("../data/interim/d2019.csv", sep = ",", index = False)
dolar_2020_txt.to_csv("../data/interim/d2020.txt", sep = " ", index = False)
dolar_2020_regex.to_csv("../data/interim/dolar_2020_excel.csv", sep = ";", index = False) 

### 6. Concatenación de DataFrames

In [13]:
df_concat_1 = pd.concat([dolar_2019, dolar_2020_txt], axis = 1) 
df_concat_0 = pd.concat([dolar_2019, dolar_2020_txt], axis = 0) 

In [14]:
df_concat_1.head()

,Día,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,...,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,year
0,1,NaN,657.81,651.79,678.53,NaN,NaN,679.15,700.82,NaN,...,852.03,NaN,806.32,821.23,NaN,776.46,788.15,NaN,767.29,2020
1,2,694.77,NaN,NaN,672.56,678.71,NaN,677.62,704.88,720.20,...,863.81,NaN,796.46,816.29,NaN,770.59,785.04,771.92,760.16,2020
2,3,697.09,NaN,NaN,670.05,683.31,709.80,681.30,NaN,724.72,...,861.89,NaN,782.86,803.98,757.06,771.45,NaN,769.17,755.34,2020
3,4,697.64,655.39,656.57,667.55,NaN,704.59,680.48,NaN,725.69,...,NaN,837.92,769.13,NaN,761.34,773.71,NaN,758.53,752.03,2020
4,5,NaN,653.80,659.07,666.32,NaN,696.78,677.64,711.13,723.30,...,NaN,838.74,775.58,NaN,767.98,NaN,790.81,757.16,NaN,2020


In [15]:
df_concat_0.head()

,Día,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,year
0,1,NaN,657.81,651.79,678.53,NaN,NaN,679.15,700.82,NaN,728.21,NaN,NaN,2019
1,2,694.77,NaN,NaN,672.56,678.71,NaN,677.62,704.88,720.20,729.38,NaN,812.13,2019
2,3,697.09,NaN,NaN,670.05,683.31,709.80,681.30,NaN,724.72,727.80,NaN,804.29,2019
3,4,697.64,655.39,656.57,667.55,NaN,704.59,680.48,NaN,725.69,719.53,735.05,802.44,2019
4,5,NaN,653.80,659.07,666.32,NaN,696.78,677.64,711.13,723.30,NaN,736.66,796.75,2019


- En df_concat_1 tendremos columnas llamadas de la misma forma (meses) lo que complicaría el análisis de un mes en particular en distintos años.
- En df_concat_0 tendremos filas con índices repetidos y que no reflejan el número real del día, lo que podría llevar a confusión al momento de analizar.